In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from PIL import Image, UnidentifiedImageError
from torchvision import transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split
from torch import nn
from torch.nn import functional as F
import random
import warnings
warnings.filterwarnings("ignore")

In [4]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.classes = ["Normal", "Porn"]
        self.image_paths = []
        self.labels = []
        self.transform = transform

        for label, cls in enumerate(self.classes):
            class_dir = os.path.join(root_dir, cls)
            for filename in os.listdir(class_dir):
                img_path = os.path.join(class_dir, filename)

                # Ensure it's an image file
                if not filename.lower().endswith(('.jpeg')):
                    continue

                # Try opening the image to check for corruption
                try:
                    with Image.open(img_path) as img:
                        img.verify()  # Verify image integrity
                    self.image_paths.append(img_path)
                    self.labels.append(label)
                except (OSError, UnidentifiedImageError):
                    print(f"Skipping corrupted image: {img_path}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        try:
            image = Image.open(image_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, label
        except (OSError, UnidentifiedImageError):
            print(f"Error loading image: {image_path}")
            return self.__getitem__((index + 1) % len(self))


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop and resize
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomGrayscale(p=0.1),  # Randomly convert to grayscale
    transforms.GaussianBlur(kernel_size=3),  # Add slight blur
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
data_path = "/content/drive/MyDrive/d/Data"
dataset = ImageDataset(data_path, transform=transform)
print("Number of training examples:", len(dataset))

Number of training examples: 39446


In [1]:
random_index = random.randint(0, len(dataset) - 1)
image, label = dataset[random_index]
plt.imshow(image.permute(1, 2, 0).numpy())
plt.title(f"Label: {'Normal' if label == 0 else 'Porn'}")
plt.show()

NameError: name 'random' is not defined

In [9]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")

Training samples: 31556, Testing samples: 7890


In [10]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 97.5MB/s]


cuda


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
import torch

num_epochs = 5
patience = 3
best_val_loss = float("inf")
counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(test_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0  # Reset counter if validation loss improves
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break  # Stop training

Error loading image: /content/drive/MyDrive/d/Data/Normal/2052.jpeg
Error loading image: /content/drive/MyDrive/d/Data/Normal/18877.jpeg
Epoch [1/5], Train Loss: 0.0368, Train Accuracy: 98.62%, Val Loss: 0.0453, Val Accuracy: 98.52%
Error loading image: /content/drive/MyDrive/d/Data/Normal/2052.jpeg
Error loading image: /content/drive/MyDrive/d/Data/Normal/18877.jpeg
Epoch [2/5], Train Loss: 0.0260, Train Accuracy: 99.10%, Val Loss: 0.0408, Val Accuracy: 98.61%
Error loading image: /content/drive/MyDrive/d/Data/Normal/2052.jpeg
Error loading image: /content/drive/MyDrive/d/Data/Normal/18877.jpeg
Epoch [3/5], Train Loss: 0.0294, Train Accuracy: 98.87%, Val Loss: 0.0334, Val Accuracy: 98.83%
Error loading image: /content/drive/MyDrive/d/Data/Normal/2052.jpeg
Error loading image: /content/drive/MyDrive/d/Data/Normal/18877.jpeg
Epoch [4/5], Train Loss: 0.0229, Train Accuracy: 99.12%, Val Loss: 0.0403, Val Accuracy: 98.76%
Error loading image: /content/drive/MyDrive/d/Data/Normal/2052.jpeg


In [14]:
model_path = "efficientnet_b0.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to efficientnet_b0.pth
